In [5]:
from boututils.datafile import DataFile
from boutdata.collect import collect
from boutdata.data import BoutData
import os, sys
import matplotlib
# matplotlib.use('Qt5Agg',force=True)
# matplotlib.use("TKAgg",force=True)
# matplotlib.use("widget",force=True)
import matplotlib.pyplot as plt
import xbout
import pandas as pd
import numpy as np
import pickle as pkl
import xarray as xr

from boutdata.restart import addvar
import glob
import shutil

import scipy

sys.path.append(r"/home/mike/work/sdtools")

from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.neutral_transport import NeutralTransport
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from hermes3.balance1d import *

# from remkit.load import load_remkit
# from code_comparison.code_comparison import *

from cycler import cycler
mpl.rcParams["axes.prop_cycle"] = cycler("color", ["teal", "darkorange", "firebrick", "limegreen", "darkorchid", "deeppink", 
                                                   "#1E90FF", "forestgreen", "#4169e1", "darkgoldenrod", "#9370db", "crimson", "#2f4f4f" ])

sys.path.append(r"/home/mike/work/ReMKiT1D_H3/scripts")
from comp_utils import H3CompFromJSON,H3nTCompPlot,H3qGCompPlot,plotTermsThatEvolveVar

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Intro
- This PR makes sure that processes that depend on collisions get the right ones
- Conduction should only take ee or ii into account
- Diffusion should only take CX and IZ into account
- Neutral viscosity should take CX and IZ 
- Ion viscosity should take ii 

# IO

In [ ]:
cs = dict()

db = CaseDB(
    case_dir = r"/home/mike/work/cases",
    grid_dir = r"/home/mike/work/cases"
    )

toload = [
    
    dict(name="zero_base", id="rkc0-oldbase_lingrid_wrongcoll_wrongcond_wrongbc", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    dict(name="zero_corrcoll", id="rkc0b-oldbase_lingrid_corrgcoll_wrongcond_wrongbc", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    
    # dict(name="add_bndry_fix", id="sc12ac-add_bndry_flux_fix", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="gamma_3.5", id="sc12ad-gamma_i_3.5", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="no_force_balance", id="sc12af-no_force_balance", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="disable_laxflux_heating", id="sc12afg-disable_laxflux_heating", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="no_vi", id="sc12b-gamma3.5_nofbalance_vi0", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="no_viscosity", id="sc12ah-disable_ion_viscosity", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    dict(name="enable_sound_speed", id="sc12ai-enable_sound_speed", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    dict(name="scale_fastestwave_0.5", id="sc12aj-scale_fastestwave_0.5", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    dict(name="scale_fastestwave_0.1", id="sc12aja-scale_fastestwave_0.1", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    dict(name="scale_fastestwave_0.05", id="sc12ajb-scale_fastestwave_0.05", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    dict(name="scale_fastestwave_0.0", id="sc12ajc-scale_fastestwave_0.0", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    # dict(name="laxon", id="sc13b-forcebalance_on_scalewave_1", unnormalise_geom = True, use_xhermes = True, guard_replace = False),  
    # dict(name="laxflux_x0.05", id="sc12b-forcebalance_on_scalewave_0.05", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxflux_off", id="sc12ba-forcebalance_on_scalewave_0.0", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    
    # dict(name="laxflux_off_fixflux_off", id="sc12baa-forcebalance_on_scalewave_0.0_fixfluxoff", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxon_fixflux_off", id="sc13bb-laxfluxon_fixfluxoff", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    
    # dict(name="laxon_fixfluxoff_loggrid", id="sc13bc-laxfluxon_fixfluxoff_loggrid", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    # dict(name="laxon_fixfluxoff_loggrid_freeNe", id="sc13bcd-laxfluxon_fixfluxoff_loggrid_free_ne", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxon_fixfluxoff_loggrid_freeNePe", id="sc13bce-laxfluxon_fixfluxoff_loggrid_free_ne_pe", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxon_fixfluxoff_loggrid_LinearFreeNePe", id="sc13bcf-laxfluxon_fixfluxoff_loggrid_linearfree_ne_pe", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    
    # dict(name="laxon_fixfluxon_loggrid_freeNePe", id="sc13bdb-laxfluxon_fixfluxon_loggrid_free_ne_pe", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxon_fixfluxon_loggrid_LinearFreeNePe", id="sc13bdc-laxfluxon_fixfluxon_loggrid_linearfree_ne_pe", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    
    # dict(name="laxon_fixfluxon", id="sc13b-forcebalance_on_scalewave_1", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxon_fixfluxon_freeNePe", id="sc14a-laxflux_fixfluxon_free_all", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="lax0.05_fixfluxon_freeNePe", id="sc14ac-laxflux0.05_fixfluxon_free_all", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxon_fixfluxoff_freeNePe", id="sc14ab-laxflux_fixfluxoff_free_all", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    # dict(name="laxon_fixfluxon_loggrid", id="sc13bc-laxfluxon_fixfluxoff_loggrid", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    
    dict(name="laxon_fixfluxon", id="sc13b-forcebalance_on_scalewave_1", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    dict(name="laxon_fixfluxon_freeNePe", id="sc14a-laxflux_fixfluxon_free_all", unnormalise_geom = True, use_xhermes = True, guard_replace = False), 
    dict(name="laxon_fixfluxon_LinFreeNePe", id="sc15a-laxflux_fixfluxon_linear_free_all", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    
    # Adding complexity
    # dict(name="old_base", id="sc16b-fix_sources_fix_lofidelity_base", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    # dict(name="old_add_qei", id="sc17a-sc16b_add_Qei", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    # dict(name="old_add_neutrals", id="sc18a-Qei_add_neutrals", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    
    # dict(name="base", id="rkc1-base_from_sc16b", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    # dict(name="add_qei", id="rkc2-Qei_from_sc17a", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
    # dict(name="add_neutrals", id="rkc3-add_neutrals_from_sc18a", unnormalise_geom = True, use_xhermes = True, guard_replace = False),
]
#
for case in toload:
    print(f"Loading case {case['name']} from {case['id']}")
    cs[case["name"]] = Load.case_1D(db.casepaths[case["id"]], guard_replace = case["guard_replace"])
    cs[case["name"]].ds.load()


Loading case zero_base from rkc0-oldbase_lingrid_wrongcoll_wrongcond_wrongbc
Loading case zero_corrcoll from rkc0b-oldbase_lingrid_corrgcoll_wrongcond_wrongbc
Loading case enable_sound_speed from sc12ai-enable_sound_speed
Loading case scale_fastestwave_0.5 from sc12aj-scale_fastestwave_0.5


# Read remkit

In [6]:
from comp_utils import H3CompFromJSON,H3nTCompPlot,H3qGCompPlot,plotTermsThatEvolveVar
import os 

In [ ]:
H3_run = "sc18a-Qei_add_neutrals"
H3Data,rk = H3CompFromJSON(filepath="./"+H3_run+".json",initialTimestep=500,solverRelTol=1e-8,solverAbsTol=1e4,convergenceVars=["density","energyDensity"],neutralTemperatureRecycled=2.0)

loadedVars = rk.loadSimulation()
dataset = loadedVars.dataset
dataset

In [3]:
rkcs = {}
wrappers = {}

# rkcs["restart"] = load_remkit(  
#                    hdf5Filepath = "/ssd_scratch/compare_data/rk2_restart_from_sc12ac",
#                    wrapperPickleFilename = "/ssd_scratch/compare_data/rk2_restart_from_sc12ac/RMK_hermes3-sc12ac-add_bndry_flux_fix.pkl",
#                    numFiles = 2
#                     )

# name = "bndry_flux_fix"
# wrappers[name], rkcs[name] = load_remkit(  
#                    hdf5Filepath = "/ssd_scratch/compare_data/RMK_hermes3-add_bndry_flux_fix/RMK_hermes3-add_bndry_flux_fix",
#                    wrapperPickleFilename = "/ssd_scratch/compare_data/RMK_hermes3-add_bndry_flux_fix/RMK_hermes3-sc12ac-add_bndry_flux_fix.pkl",
#                    numFiles = 8
#                     )

# name = "gamma_3.5"
# wrappers[name], rkcs[name] = load_remkit(  
#                    hdf5Filepath = "/ssd_scratch/compare_data/RMK_hermes3-sc12ac-add_bndry_flux_fix-no-kin/RMK_hermes3-add_bndry_flux_fix",
#                    wrapperPickleFilename = "/ssd_scratch/compare_data/RMK_hermes3-sc12ac-add_bndry_flux_fix-no-kin/RMK_hermes3-sc12ac-add_bndry_flux_fix-no-kin.pkl",
#                    numFiles = 8
#                     )

# name = "lofidelity_base"
# wrappers[name], rkcs[name] = load_remkit(  
#                    hdf5Filepath = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc13bcf-lofidelitybase/RMK_hermes3-sc13bcf-laxfluxon_fixfluxoff_loggrid_linearfree_ne_pe",
#                    wrapperPickleFilename = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc13bcf-lofidelitybase/RMK_hermes3-sc13bcf-laxfluxon_fixfluxoff_loggrid_linearfree_ne_pe.pkl",
#                    numFiles = 8
#                     )

name = "lofidelity_base_reggrid"
wrappers[name], rkcs[name] = load_remkit(  
                   hdf5Filepath = "/home/mike/work/compare_data/remkit-cases/RMK_hermes3-sc14ad-lofidelitybase_reggrid/RMK_hermes3-sc14ad-laxfluxon_fixfluxoff_reggrid_linearfree_ne_pe",
                   wrapperPickleFilename = "/home/mike/work/compare_data/remkit-cases/RMK_hermes3-sc14ad-lofidelitybase_reggrid/RMK_hermes3-sc14ad-laxfluxon_fixfluxoff_reggrid_linearfree_ne_pe.pkl",
                   numFiles = 8
                    )


# name = "fix_sources"
# wrappers[name], rkcs[name] = load_remkit(  
#                    hdf5Filepath = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc14ad-lofidelitybase_reggrid/RMK_hermes3-sc14ad-laxfluxon_fixfluxoff_reggrid_linearfree_ne_pe",
#                    wrapperPickleFilename = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc14ad-lofidelitybase_reggrid/RMK_hermes3-sc14ad-laxfluxon_fixfluxoff_reggrid_linearfree_ne_pe.pkl",
#                    numFiles = 8
#                     )

# name = "add_Qei"
# wrappers[name], rkcs[name] = load_remkit(  
#                    hdf5Filepath = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc17a-sc16b_add_Qei/RMK_hermes3-sc17a-sc16b_add_Qei",
#                    wrapperPickleFilename = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc17a-sc16b_add_Qei/RMK_hermes3-sc17a-sc16b_add_Qei.pkl",
#                    numFiles = 8
#                     )

# name = "neutrals"
# wrappers[name], rkcs[name] = load_remkit(  
#                    hdf5Filepath = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc18a-Qei_add_neutrals/RMK_hermes3-sc18a-Qei_add_neutrals",
#                    wrapperPickleFilename = "/ssd_scratch/compare_data/remkit_comparison/RMK_hermes3-sc18a-Qei_add_neutrals/RMK_hermes3-sc18a-Qei_add_neutrals.pkl",
#                    numFiles = 8
#                     )

NameError: name 'load_remkit' is not defined

# Fixing collisionality choice and conduction